# SDA - lecture 2 - Stochastic processes

In [ ]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

%matplotlib inline

### Calculate the firing rate of neurons

In [ ]:
# Generate a "spike train" of a simulated Poisson neuron 

samp = 1000
rate = 20 / samp
duration = 5

spk_array = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
time_array = np.arange(0, duration, 1/samp)

def rate_plot(x, y, ax, title, samp=1000):
    ax.plot(x,y*samp)
    ax.set_ylabel('Firing rate\n[sp/s]')
    ax.set_title(title)
    ax.grid()
    
fig, ax = plt.subplots(figsize=(12,20), nrows=5, ncols=1)

rate_plot(time_array, spk_array, ax[0], 'Spike train')

bin_size = 100
bin_array = np.zeros_like(spk_array, dtype=np.float32)
for i in np.arange(0, samp*duration, bin_size):
    bin_array[i:i+bin_size] = spk_array[i:i+bin_size].sum() / bin_size
rate_plot(time_array, bin_array, ax[1], 'Binned Spike train')

win_size = 100
win_array = np.ones(win_size)/win_size
rec_conv_array = np.convolve(spk_array, win_array, 'same')
rate_plot(time_array, rec_conv_array, ax[2], 'Rectengular convolution')

win_size = 200
conv_std = 50
gauss_array = sig.gaussian(win_size, conv_std) 
gauss_conv_array = np.convolve(spk_array, gauss_array, 'same')
rate_plot(time_array, gauss_conv_array, ax[3], 'Gaussian convolution')

alpha = 0.02
t = np.arange(-300,301)
alpha_array = alpha**2*(t*(t>0))*np.exp(-alpha*t)
alpha_conv_array = np.convolve(spk_array, alpha_array, 'same')
rate_plot(time_array, alpha_conv_array, ax[4], 'Causal convolution')

ax[4].set_xlabel('Time [s]');

### Multiple spike trains of the same/different neuron

In [ ]:
nex = 11
fig, ax = plt.subplots(figsize=(6,6), nrows=nex, ncols=1)
def plot_pts(t, ax, c, rng):
    ax.plot(t, np.zeros_like(t),'.'+c)  
    ax.tick_params(left=False,
                bottom=False,
                labelleft=False,
                labelbottom=False)
    ax.set_xlim(rng)

samp = 1000
duration = 2
rate1, rate2 = 5/samp, 5/samp

for i in range(nex):
    if i<nex-1:
        rate, c = rate1, 'b'
    else:
        rate, c = rate2, 'r'
    spk_time = np.nonzero(np.random.uniform(size=samp*duration)<rate)
    plot_pts(spk_time,ax[i],c,[0, samp*duration])

### Signal to noise ratio

In [ ]:
def rms(s):
    return np.sqrt(np.mean(s**2))

samp = 1000
noise = np.random.normal(scale=2,size=samp)
signal = np.sin(np.arange(0,1,1/samp)*2*np.pi)*3
fig, ax = plt.subplots(figsize=(12,4), nrows=1, ncols=3)
ax[0].plot(signal)
ax[1].plot(noise)
ax[2].plot(signal+noise)

rms_signal, rms_noise = rms(signal), rms(noise)
print(f'RMS: signal {rms_signal:.2f} noise {rms_noise:.2f}')
snr = rms_signal / rms_noise
print(f'SNR: {snr:.2f} or {20*np.log10(snr):.2f}dB')

